<a href="https://colab.research.google.com/github/NataliaGon/kpi/blob/AI-cybersecurity/Lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Метод до чутливого розташування хешування в системах машинного навчання

In [22]:
import os
import numpy as np
import pandas as pd
import kagglehub
from PIL import Image
from tqdm import tqdm
from numpy.linalg import norm
from collections import Counter
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score


In [23]:
path = kagglehub.dataset_download("phucthaiv02/butterfly-image-classification")


print("Path to dataset files:", path)

print("Files in dataset folder:", os.listdir(path))

Path to dataset files: /root/.cache/kagglehub/datasets/phucthaiv02/butterfly-image-classification/versions/3
Files in dataset folder: ['Testing_set.csv', 'test', 'train', 'Training_set.csv']


In [24]:
base_path = "/root/.cache/kagglehub/datasets/phucthaiv02/butterfly-image-classification/versions/3"

train_csv = pd.read_csv(os.path.join(base_path, "Training_set.csv"))
train_dir = os.path.join(base_path, "train")

print("Train samples in CSV:", len(train_csv))

Train samples in CSV: 6499


In [25]:
print("TRAIN columns:", train_csv.columns.tolist())
print(train_csv.head())


TRAIN columns: ['filename', 'label']
      filename                     label
0  Image_1.jpg          SOUTHERN DOGFACE
1  Image_2.jpg                    ADONIS
2  Image_3.jpg            BROWN SIPROETA
3  Image_4.jpg                   MONARCH
4  Image_5.jpg  GREEN CELLED CATTLEHEART


In [27]:
X = []
y = []
X_test = []
y_test = []

def load_images_labeled(base_dir, csv_file):
    X, y = [], []
    for _, row in tqdm(csv_file.iterrows(), total=len(csv_file)):
        img_path = os.path.join(base_dir, row["filename"])
        if not os.path.isfile(img_path):
            continue
        img = Image.open(img_path).convert("RGB").resize((64, 64))
        arr = np.array(img, dtype="float32") / 255.0
        X.append(arr.flatten())
        y.append(row["label"])
    return np.array(X), np.array(y)


X, y = load_images_labeled(train_dir, train_csv)
print("All train shape:", X.shape, "classes:", len(np.unique(y)))

X_train, X_test, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print("Train:", X_train.shape, "Val:", X_test.shape)


100%|██████████| 6499/6499 [00:12<00:00, 529.76it/s]


All train shape: (6499, 12288) classes: 75
Train: (5199, 12288) Val: (1300, 12288)


**Locality Sensitive Hashing (LSH)**

In [28]:
class LSH:
    def __init__(self, num_buckets=1000, num_hash_functions=10, input_dim=128, seed=42):
        self.num_buckets = num_buckets
        self.num_hash_functions = num_hash_functions
        self.input_dim = input_dim
        rng = np.random.default_rng(seed)
        # кожна хеш-функція — це гіперплощина
        self.hash_functions = [rng.standard_normal(input_dim) for _ in range(num_hash_functions)]
        # робимо словник замість фіксованого масиву
        self.buckets = defaultdict(list)

    def _hash_vector(self, vector):
        # отримаємо бітовий підпис, напр. [1,0,1,1,0,...]
        bits = []
        for h in self.hash_functions:
            proj = np.dot(vector, h)
            bits.append(1 if proj > 0 else 0)
        # перетворимо біти в одне число
        hval = 0
        for b in bits:
            hval = (hval << 1) | b
        return hval % self.num_buckets

    def add_vector(self, idx, vector):
        h = self._hash_vector(vector)
        self.buckets[h].append(idx)

    def query(self, query_vector):
        h = self._hash_vector(query_vector)
        # повертаємо кандидатів (індекси)
        return self.buckets.get(h, [])


In [30]:

# 7. build LSH on OUR train part
lsh = LSH(num_buckets=2048,
          num_hash_functions=12,
          input_dim=X_train.shape[1])

for i, v in enumerate(X_train):
    lsh.add_vector(i, v)

# 8. evaluate LSH on validation part
def cosine_sim(a, b):
    return np.dot(a, b) / (norm(a) * norm(b) + 1e-8)

y_pred_lsh = []
y_true_lsh = []

for i in range(len(X_test)):
    q = X_test[i]
    true_label = y_val[i]

    cand_idx = lsh.query(q)
    if not cand_idx:
        continue  # no candidates in that bucket

    # majority in bucket (simple)
    cand_labels = [y_train[j] for j in cand_idx]
    most_common_label, _ = Counter(cand_labels).most_common(1)[0]

    y_true_lsh.append(true_label)
    y_pred_lsh.append(most_common_label)

print("LSH accuracy (validation, on found):",
      accuracy_score(y_true_lsh, y_pred_lsh))
print(classification_report(y_true_lsh, y_pred_lsh))
print("Used", len(y_true_lsh), "val images out of", len(X_test))




LSH accuracy (validation, on found): 0.029526029526029528
                           precision    recall  f1-score   support

                   ADONIS       0.01      0.06      0.01        18
AFRICAN GIANT SWALLOWTAIL       0.03      0.33      0.05        15
           AMERICAN SNOOT       0.00      0.00      0.00        14
                    AN 88       0.00      0.00      0.00        17
                  APPOLLO       0.00      0.00      0.00        18
                    ATALA       0.14      0.05      0.07        20
 BANDED ORANGE HELICONIAN       0.00      0.00      0.00        20
           BANDED PEACOCK       0.00      0.00      0.00        17
            BECKERS WHITE       0.00      0.00      0.00        15
         BLACK HAIRSTREAK       0.00      0.00      0.00        17
              BLUE MORPHO       0.00      0.00      0.00        15
        BLUE SPOTTED CROW       0.00      0.00      0.00        17
           BROWN SIPROETA       0.00      0.00      0.00        20
   

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**Logistic regression**

In [31]:
clf = LogisticRegression(max_iter=500)
clf.fit(X_train, y_train)
y_pred_val = clf.predict(X_test)

print("Logistic Regression accuracy:", accuracy_score(y_val, y_pred_val))


Logistic Regression accuracy: 0.3230769230769231


**Висновки**

У цій роботі я взяла датасет з Kaggle (Butterfly Image Classification) і перетворила зображення у числові вектори фіксованого розміру 64×64×3 = 12 288 ознак. На основі цих ознак ти побудувала власний train–test split (80/20), бо в наданому Kaggle-тесті не було міток, тобто його не можна було використати для оцінювання. Це правильне рішення, бо інакше порівняння LSH із класичною класифікацією було б неможливим.

Метод LSH реалізовано на основі випадкових гіперплощин, які формують бітові підписи векторів зображень і розподіляють їх по «бакетах» за схожістю. Для кожного зображення тестової вибірки здійснювався пошук схожих елементів у відповідному бакеті, а прогнозована мітка визначалася за принципом більшості серед знайдених сусідів. Як базовий підхід для порівняння використано метод логістичної регресії, натренований на тих самих даних.

Отримані результати показали, що LSH продемонстрував значно нижчу точність класифікації порівняно з логістичною регресією. Це пояснюється тим, що LSH є наближеним методом, оптимізованим для швидкого пошуку подібних об’єктів у високовимірних просторах, а не для точного багатокласового навчання на сирих піксельних даних. Попри це, експеримент підтвердив здатність LSH ефективно скорочувати простір пошуку та може бути корисним як попередній етап для відбору кандидатів у системах обробки великих обсягів зображень. Таким чином, робота продемонструвала практичне застосування LSH у системах машинного навчання та його відмінності від традиційних методів класифікації за точністю та обчислювальною складністю.